1. **Is it OK to initialize all the weights to the same value as long as that value is selected randomly using He initialization?**
   - No, initializing all weights to the same value, even if it's randomly chosen using He initialization, is not advisable. The purpose of He initialization is to break the symmetry between neurons in the same layer. Initializing all weights to the same value would result in the same gradients during training, making it challenging for the network to learn effectively.



2. **Is it OK to initialize the bias terms to 0?**
   - Yes, we can initialize bias terms to 0. In many cases, bias terms are initialized to zero, and the network learns the appropriate bias values during training. However, there are variations where bias terms can be initialized differently for specific reasons.


3. **Name three advantages of the SELU activation function over ReLU.**
   - Advantages of the SELU activation function over ReLU:
     - Self-normalization: SELU is designed to maintain the mean and variance of neuron activations close to specific values, which helps address the vanishing/exploding gradient problem.
     - Continuous and smooth: SELU is a smooth, non-monotonic function, making it differentiable everywhere. This allows for gradient-based optimization.
     - Improved training performance: SELU often leads to faster convergence and better generalization compared to ReLU in deep networks when the self-normalization property holds.


4. **In which cases would you want to use each of the following activation functions: SELU, leaky ReLU (and its variants), ReLU, tanh, logistic, and softmax?**
   - SELU: Suitable for deep neural networks, particularly when self-normalization is desired.
   - Leaky ReLU: A good choice for deep networks where gradient vanishing is a concern. Variants like Parametric Leaky ReLU (PReLU) can adaptively learn the slope.
   - ReLU: Widely used in many cases due to its simplicity and effectiveness. Suitable for feed-forward networks, but can lead to dead neurons.
   - Tanh: Suitable for feed-forward networks where outputs need to be centered around zero. Often used in recurrent neural networks (RNNs).
   - Logistic (Sigmoid): Used in binary classification output layers. Maps values to a range between 0 and 1.
   - Softmax: Used in multi-class classification output layers. Converts raw scores into class probabilities.



5. **What may happen if you set the momentum hyperparameter too close to 1 (e.g., 0.99999) when using an SGD optimizer?**
   - Setting the momentum hyperparameter very close to 1 in stochastic gradient descent (SGD) can result in slow convergence and potential instability. It will make the optimization process heavily biased towards past gradients and may not react quickly to new information. This can lead to slow training, and in extreme cases, the optimization process may diverge.


6. **Name three ways you can produce a sparse model.**
   - Three ways to produce a sparse model are:
     - L1 Regularization (Lasso): Regularize the model using L1 regularization, which encourages many weights to be exactly zero.
     - Pruning: Prune unimportant or low-magnitude weights after training. This reduces the number of active connections in the network.
     - DropConnect: Similar to dropout but randomly sets entire connections (weights) to zero during training.


7. **Does dropout slow down training? Does it slow down inference (i.e., making predictions on new instances)? What about MC Dropout?**
   - Dropout can slow down training because it introduces randomness by temporarily disabling a random subset of neurons during each forward and backward pass. However, this regularization technique can improve model generalization. During inference (making predictions on new data), dropout is turned off, so it doesn't slow down inference.
   - MC Dropout (Monte Carlo Dropout) involves running the model multiple times with dropout enabled during inference to obtain more stable and calibrated predictions. This does slow down inference significantly as it involves multiple passes through the network.


**8. Practice training a deep neural network on the CIFAR10 image dataset:**

**a ) Build a DNN with 20 hidden layers of 100 neurons each using He initialization and the ELU activation function.**

```python
import tensorflow as tf
from tensorflow import keras

# load the CIFAR10 dataset
(X_train_full, y_train_full), (X_test, y_test) = keras.datasets.cifar10.load_data()

# normalize the pixel values
X_train_full, X_test = X_train_full / 255.0, X_test / 255.0

# build a deep neural network
model = keras.models.Sequential()
model.add(keras.layers.Flatten(input_shape=[32, 32, 3]))

for _ in range(20):
    model.add(keras.layers.Dense(100, activation="elu", kernel_initializer="he_normal"))

model.add(keras.layers.Dense(10, activation="softmax"))

# compile the model
model.compile(loss="sparse_categorical_crossentropy", optimizer="nadam", metrics=["accuracy"])
```

**b) Train the network on the CIFAR10 dataset using Nadam optimization and early stopping.**

```python
# early stopping callback
early_stopping_cb = keras.callbacks.EarlyStopping(patience=10, restore_best_weights=True)

# train the model
history = model.fit(X_train_full, y_train_full, epochs=100, validation_split=0.1, callbacks=[early_stopping_cb])

# evaluate the model on the test set
test_loss, test_accuracy = model.evaluate(X_test, y_test)
print(f"Test accuracy: {test_accuracy * 100:.2f}%")
```


**c) Add Batch Normalization and compare learning curves.**

We can add Batch Normalization layers to the model and observe how it affects training speed and convergence.

```python
# build a DNN with Batch Normalization
model = keras.models.Sequential()
model.add(keras.layers.Flatten(input_shape=[32, 32, 3]))

for _ in range(20):
    model.add(keras.layers.Dense(100, kernel_initializer="he_normal", use_bias=False))
    model.add(keras.layers.BatchNormalization())
    model.add(keras.layers.Activation("elu"))

model.add(keras.layers.Dense(10, activation="softmax"))

# early stopping callback
early_stopping_cb = keras.callbacks.EarlyStopping(patience=10, restore_best_weights=True)

# train the model
history = model.fit(X_train_full, y_train_full, epochs=100, validation_split=0.1, callbacks=[early_stopping_cb])

# evaluate the model on the test set
test_loss, test_accuracy = model.evaluate(X_test, y_test)
print(f"Test accuracy: {test_accuracy * 100:.2f}%")
```


**d) Replace Batch Normalization with SELU and make the necessary adjustments.**


```python
# Build a DNN with SELU activation and LeCun initialization
model = keras.models.Sequential()
model.add(keras.layers.Flatten(input_shape=[32, 32, 3]))

for _ in range(20):
    model.add(keras.layers.Dense(100, activation="selu", kernel_initializer="lecun_normal"))

model.add(keras.layers.Dense(10, activation="softmax"))


# early stopping callback
early_stopping_cb = keras.callbacks.EarlyStopping(patience=10, restore_best_weights=True)

# train the model
history = model.fit(X_train_full, y_train_full, epochs=100, validation_split=0.1, callbacks=[early_stopping_cb])

# evaluate the model on the test set
test_loss, test_accuracy = model.evaluate(X_test, y_test)
print(f"Test accuracy: {test_accuracy * 100:.2f}%")
```


**e) Try regularizing the model with alpha dropout and MC Dropout.**
```python
# regularize the model with alpha dropout
model = keras.models.Sequential()
model.add(keras.layers.Flatten(input_shape=[32, 32, 3]))

for _ in range(20):
    model.add(keras.layers.Dense(100, activation="selu", kernel_initializer="lecun_normal"))
    model.add(keras.layers.AlphaDropout(rate=0.5))

model.add(keras.layers.Dense(10, activation="softmax"))

# early stopping callback
early_stopping_cb = keras.callbacks.EarlyStopping(patience=10, restore_best_weights=True)

# train the model
history = model.fit(X_train_full, y_train_full, epochs=100, validation_split=0.1, callbacks=[early_stopping_cb])

# evaluate the model on the test set
test_loss, test_accuracy = model.evaluate(X_test, y_test)
print(f"Test accuracy: {test_accuracy * 100:.2f}%")

# MC model
mc_model = keras.models.Sequential([model, keras.layers.Dropout(rate=0.5)])

# run MC Dropout to make predictions and estimate uncertainty
predictions = [mc_model.predict(X_test) for _ in range(100)]  # Adjust the number of runs as needed
predictions_mean = np.mean(predictions, axis=0)
predictions_std = np.std(predictions, axis=0)
```
